In [2]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import joblib
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.animation import FuncAnimation
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# API Keys - Set these as environment variables or replace with your actual keys
ALPHA_VANTAGE_API_KEY = 'SHBVCS0U37OTIADV'  # Replace with your actual key
MARKETAUX_API_KEY = '9zeMnYnU8JAqXgxQAqm0KWN8vZGn9vnV2Q91AeRY'  # Replace with your actual key

# Indian stock ticker mappings (common ones)
INDIAN_STOCKS = {
    'reliance': 'RELIANCE.BSE',
    'tcs': 'TCS.BSE',
    'infosys': 'INFY.BSE',
    'hdfc bank': 'HDFCBANK.BSE',
    'icici bank': 'ICICIBANK.BSE',
    'wipro': 'WIPRO.BSE',
    'bharti airtel': 'BHARTIARTL.BSE',
    'itc': 'ITC.BSE',
    'hul': 'HINDUNILVR.BSE',
    'hindustan unilever': 'HINDUNILVR.BSE',
    'maruti suzuki': 'MARUTI.BSE',
    'bajaj finance': 'BAJFINANCE.BSE',
    'axis bank': 'AXISBANK.BSE',
    'kotak mahindra': 'KOTAKBANK.BSE',
    'l&t': 'LT.BSE',
    'larsen & toubro': 'LT.BSE',
    'sun pharma': 'SUNPHARMA.BSE',
    'ntpc': 'NTPC.BSE',
    'power grid': 'POWERGRID.BSE',
    'ongc': 'ONGC.BSE',
    'tatasteel': 'TATASTEEL.BSE',
    'tata steel': 'TATASTEEL.BSE',
    'adani enterprises': 'ADANIENT.BSE',
    'adani ports': 'ADANIPORTS.BSE'
}

# US stock ticker mappings (common ones)
US_STOCKS = {
    'apple': 'AAPL',
    'microsoft': 'MSFT',
    'google': 'GOOGL',
    'alphabet': 'GOOGL',
    'amazon': 'AMZN',
    'tesla': 'TSLA',
    'meta': 'META',
    'facebook': 'META',
    'nvidia': 'NVDA',
    'netflix': 'NFLX',
    'disney': 'DIS',
    'coca cola': 'KO',
    'pepsi': 'PEP',
    'walmart': 'WMT',
    'johnson & johnson': 'JNJ',
    'procter & gamble': 'PG',
    'visa': 'V',
    'mastercard': 'MA',
    'jpmorgan': 'JPM',
    'goldman sachs': 'GS',
    'intel': 'INTC',
    'cisco': 'CSCO',
    'oracle': 'ORCL',
    'ibm': 'IBM',
    'adobe': 'ADBE',
    'salesforce': 'CRM',
    'uber': 'UBER',
    'airbnb': 'ABNB',
    'spotify': 'SPOT',
    'zoom': 'ZM',
    'twitter': 'TWTR',
    'snap': 'SNAP',
    'pinterest': 'PINS',
    'paypal': 'PYPL',
    'square': 'SQ',
    'robinhood': 'HOOD',
    'coinbase': 'COIN'
}

def get_ticker_by_name(company_name):
    """Get stock ticker from company name using local mappings"""
    company_lower = company_name.lower().strip()
    
    # Check Indian stocks first
    if company_lower in INDIAN_STOCKS:
        return INDIAN_STOCKS[company_lower], 'Indian'
    
    # Check US stocks
    if company_lower in US_STOCKS:
        return US_STOCKS[company_lower], 'US'
    
    # Try partial matches for Indian stocks
    for key, value in INDIAN_STOCKS.items():
        if company_lower in key or key in company_lower:
            return value, 'Indian'
    
    # Try partial matches for US stocks
    for key, value in US_STOCKS.items():
        if company_lower in key or key in company_lower:
            return value, 'US'
    
    return None, None

def validate_ticker_symbol(symbol):
    """Validate if the ticker symbol exists by making a test API call"""
    params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': symbol,
        'apikey': ALPHA_VANTAGE_API_KEY,
        'outputsize': 'compact'
    }
    
    try:
        response = requests.get('https://www.alphavantage.co/query', params=params, timeout=10)
        data = response.json()
        
        # Check for API errors
        if 'Error Message' in data:
            return False
        if 'Note' in data:
            print("⚠️  API call frequency limit reached. Please wait a moment.")
            return False
        if 'Time Series (Daily)' not in data:
            return False
        
        return True
        
    except Exception as e:
        print(f"Error validating ticker: {e}")
        return False

def get_alpha_vantage_data(symbol, days=90):
    """Get stock price data from Alpha Vantage"""
    params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': symbol,
        'apikey': ALPHA_VANTAGE_API_KEY,
        'outputsize': 'full'
    }
    
    try:
        response = requests.get('https://www.alphavantage.co/query', params=params, timeout=30)
        
        if response.status_code != 200:
            raise ValueError(f"API request failed with status {response.status_code}")
        
        data = response.json()
        
        # Check for API errors
        if 'Error Message' in data:
            raise ValueError(f"API Error: {data['Error Message']}")
        if 'Note' in data:
            raise ValueError("API call frequency limit reached. Please wait 1 minute and try again.")
        if 'Time Series (Daily)' not in data:
            raise ValueError('Invalid ticker symbol or API response')
        
        # Convert to DataFrame
        ts_data = data['Time Series (Daily)']
        df = pd.DataFrame.from_dict(ts_data, orient='index')
        df = df.astype(float)
        df.columns = ['open', 'high', 'low', 'close', 'volume']
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()
        df = df.tail(days)
        
        return df
        
    except requests.RequestException as e:
        raise ValueError(f"Network error: {e}")

def get_marketaux_sentiment(symbol, days=7):
    """Get recent news sentiment for the stock"""
    # Clean symbol for news search (remove exchange suffix)
    clean_symbol = symbol.split('.')[0]
    
    # Get news from last week
    from_date = (datetime.now() - timedelta(days=days)).strftime('%Y-%m-%d')
    
    params = {
        'api_token': MARKETAUX_API_KEY,
        'symbols': clean_symbol,
        'filter_entities': True,
        'published_after': from_date,
        'limit': 50
    }
    
    try:
        response = requests.get('https://api.marketaux.com/v1/news/all', params=params, timeout=30)
        
        if response.status_code != 200:
            print(f"News API warning: Status {response.status_code}")
            return None
            
        data = response.json()
        return data
        
    except requests.RequestException as e:
        print(f"Error getting news data: {e}")
        return None

def calculate_technical_indicators(df):
    """Calculate technical indicators for the stock data"""
    # Create a copy to avoid modifying original
    df = df.copy()
    
    # Price-based indicators
    df['price_change'] = df['close'].pct_change()
    df['price_change_2d'] = df['close'].pct_change(periods=2)
    df['price_change_5d'] = df['close'].pct_change(periods=5)
    
    # Moving averages
    df['ma_5'] = df['close'].rolling(window=5).mean()
    df['ma_10'] = df['close'].rolling(window=10).mean()
    df['ma_20'] = df['close'].rolling(window=20).mean()
    
    # Moving average signals
    df['ma_5_signal'] = (df['close'] > df['ma_5']).astype(int)
    df['ma_10_signal'] = (df['close'] > df['ma_10']).astype(int)
    df['ma_20_signal'] = (df['close'] > df['ma_20']).astype(int)
    
    # RSI (Relative Strength Index)
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # Volume indicators
    df['volume_change'] = df['volume'].pct_change()
    df['volume_ma_5'] = df['volume'].rolling(window=5).mean()
    df['volume_ratio'] = df['volume'] / df['volume_ma_5']
    
    # Volatility
    df['volatility'] = df['close'].rolling(window=10).std()
    
    # High-Low spread
    df['hl_spread'] = (df['high'] - df['low']) / df['close']
    
    # Target variable: Next day direction (1 = up, 0 = down)
    df['next_day_direction'] = (df['close'].shift(-1) > df['close']).astype(int)
    
    return df

def analyze_sentiment(news_data):
    """Analyze sentiment from news data"""
    if not news_data or 'data' not in news_data:
        return 0.0  # Neutral sentiment if no data
    
    sentiments = []
    for article in news_data['data']:
        # Combine title and description for sentiment analysis
        text = f"{article.get('title', '')} {article.get('description', '')}"
        if text.strip():
            # Use TextBlob for simple sentiment analysis
            blob = TextBlob(text)
            sentiments.append(blob.sentiment.polarity)
    
    # Return average sentiment (-1 to 1, where 1 is most positive)
    return np.mean(sentiments) if sentiments else 0.0

def prepare_features(df, sentiment_score):
    """Prepare features for machine learning"""
    # Select relevant columns for prediction
    feature_columns = [
        'price_change', 'price_change_2d', 'price_change_5d',
        'ma_5_signal', 'ma_10_signal', 'ma_20_signal',
        'rsi', 'volume_change', 'volume_ratio',
        'volatility', 'hl_spread'
    ]
    
    # Create feature matrix
    features_df = df[feature_columns + ['next_day_direction']].copy()
    
    # Add sentiment as a feature
    features_df['sentiment'] = sentiment_score
    
    # Remove rows with NaN values
    features_df = features_df.dropna()
    
    return features_df

def train_and_predict_with_history(features_df, target_column='next_day_direction'):
    """Train Random Forest model and make predictions with historical performance"""
    # Remove the last row (we don't have tomorrow's actual result yet)
    df_for_training = features_df[:-1].copy()
    X = df_for_training.drop(columns=[target_column])
    y = df_for_training[target_column]
    
    if len(X) < 20:
        print("Not enough historical data for reliable prediction (need at least 20 days)")
        return None, None, None, None
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    # Train model
    model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight='balanced')
    model.fit(X_train, y_train)
    
    # Make predictions on test set
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"\n=== Model Performance ===")
    print(f"Training samples: {len(X_train)}")
    print(f"Test samples: {len(X_test)}")
    print(f"Accuracy: {accuracy:.2%}")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 5 Most Important Features:")
    print(feature_importance.head())
    
    # Make prediction for tomorrow
    latest_features = features_df.iloc[-1:].drop(columns=[target_column])
    tomorrow_prediction = model.predict(latest_features)[0]
    tomorrow_probability = model.predict_proba(latest_features)[0]
    
    # Create historical predictions for plotting
    historical_predictions = []
    historical_probabilities = []
    
    # Use time series cross-validation approach for more realistic historical predictions
    for i in range(20, len(X)):  # Start from day 20 to have enough training data
        X_historical = X.iloc[:i]
        y_historical = y.iloc[:i]
        
        # Train model on historical data
        temp_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight='balanced')
        temp_model.fit(X_historical, y_historical)
        
        # Predict next day
        next_day_features = X.iloc[i:i+1]
        pred = temp_model.predict(next_day_features)[0]
        pred_proba = temp_model.predict_proba(next_day_features)[0]
        
        historical_predictions.append(pred)
        historical_probabilities.append(pred_proba[1])  # Probability of going up
    
    # Create results DataFrame
    results_df = df_for_training.iloc[20:].copy()
    results_df['predicted_direction'] = historical_predictions
    results_df['predicted_probability'] = historical_probabilities
    
    return model, tomorrow_prediction, tomorrow_probability, results_df

def plot_comprehensive_analysis(stock_df, results_df, ticker, sentiment_score, market_type):
    """Create comprehensive plotting dashboard"""
    fig = plt.figure(figsize=(20, 16))
    
    # Determine currency symbol
    currency = "₹" if market_type == "Indian" else "$"
    
    # Plot 1: Stock Price with Moving Averages
    ax1 = plt.subplot(3, 3, 1)
    ax1.plot(stock_df.index, stock_df['close'], label='Close Price', linewidth=2, color='blue')
    ax1.plot(stock_df.index, stock_df['ma_5'], label='MA 5', alpha=0.7, color='red')
    ax1.plot(stock_df.index, stock_df['ma_10'], label='MA 10', alpha=0.7, color='green')
    ax1.plot(stock_df.index, stock_df['ma_20'], label='MA 20', alpha=0.7, color='orange')
    ax1.set_title(f'{ticker} - Price with Moving Averages', fontsize=14, fontweight='bold')
    ax1.set_ylabel(f'Price ({currency})')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Volume Analysis
    ax2 = plt.subplot(3, 3, 2)
    ax2.bar(stock_df.index, stock_df['volume'], alpha=0.7, color='purple')
    ax2.plot(stock_df.index, stock_df['volume_ma_5'], label='Volume MA 5', color='red', linewidth=2)
    ax2.set_title(f'{ticker} - Volume Analysis', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Volume')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: RSI
    ax3 = plt.subplot(3, 3, 3)
    ax3.plot(stock_df.index, stock_df['rsi'], label='RSI', color='purple', linewidth=2)
    ax3.axhline(y=70, color='red', linestyle='--', alpha=0.7, label='Overbought (70)')
    ax3.axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Oversold (30)')
    ax3.set_title(f'{ticker} - RSI Indicator', fontsize=14, fontweight='bold')
    ax3.set_ylabel('RSI')
    ax3.set_ylim(0, 100)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Actual vs Predicted Directions
    ax4 = plt.subplot(3, 3, 4)
    dates = results_df.index
    actual_directions = results_df['next_day_direction'].values
    predicted_directions = results_df['predicted_direction'].values
    
    # Create comparison plot
    x_pos = np.arange(len(dates))
    width = 0.35
    
    ax4.bar(x_pos - width/2, actual_directions, width, label='Actual Direction', alpha=0.7, color='blue')
    ax4.bar(x_pos + width/2, predicted_directions, width, label='Predicted Direction', alpha=0.7, color='red')
    
    ax4.set_title('Actual vs Predicted Directions', fontsize=14, fontweight='bold')
    ax4.set_ylabel('Direction (0=Down, 1=Up)')
    ax4.set_xlabel('Trading Days')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # Plot 5: Prediction Confidence Over Time
    ax5 = plt.subplot(3, 3, 5)
    ax5.plot(dates, results_df['predicted_probability'], label='Prediction Confidence', color='green', linewidth=2)
    ax5.axhline(y=0.5, color='black', linestyle='--', alpha=0.5, label='Neutral (50%)')
    ax5.set_title('Prediction Confidence Over Time', fontsize=14, fontweight='bold')
    ax5.set_ylabel('Probability of Price Increase')
    ax5.set_ylim(0, 1)
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    # Plot 6: Price Change Distribution
    ax6 = plt.subplot(3, 3, 6)
    ax6.hist(stock_df['price_change'].dropna(), bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    ax6.set_title('Price Change Distribution', fontsize=14, fontweight='bold')
    ax6.set_xlabel('Daily Price Change (%)')
    ax6.set_ylabel('Frequency')
    ax6.grid(True, alpha=0.3)
    
    # Plot 7: Accuracy Analysis
    ax7 = plt.subplot(3, 3, 7)
    correct_predictions = (results_df['next_day_direction'] == results_df['predicted_direction']).astype(int)
    rolling_accuracy = pd.Series(correct_predictions).rolling(window=10).mean()
    
    ax7.plot(dates, rolling_accuracy, label='10-Day Rolling Accuracy', color='purple', linewidth=2)
    ax7.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Random Chance (50%)')
    ax7.set_title('Model Accuracy Over Time', fontsize=14, fontweight='bold')
    ax7.set_ylabel('Accuracy')
    ax7.set_ylim(0, 1)
    ax7.legend()
    ax7.grid(True, alpha=0.3)
    
    # Plot 8: Volatility Analysis
    ax8 = plt.subplot(3, 3, 8)
    ax8.plot(stock_df.index, stock_df['volatility'], label='Price Volatility', color='orange', linewidth=2)
    ax8.set_title('Price Volatility Over Time', fontsize=14, fontweight='bold')
    ax8.set_ylabel('Volatility')
    ax8.legend()
    ax8.grid(True, alpha=0.3)
    
    # Plot 9: Sentiment and Performance Summary
    ax9 = plt.subplot(3, 3, 9)
    
    # Calculate some summary statistics
    total_predictions = len(results_df)
    correct_predictions = (results_df['next_day_direction'] == results_df['predicted_direction']).sum()
    accuracy = correct_predictions / total_predictions
    
    # Create text summary
    summary_text = f"""
    PREDICTION SUMMARY
    ==================
    
    Ticker: {ticker}
    Market: {market_type}
    Currency: {currency}
    Total Predictions: {total_predictions}
    Correct Predictions: {correct_predictions}
    Accuracy: {accuracy:.2%}
    
    Current Sentiment: {sentiment_score:.3f}
    Latest Close: {currency}{stock_df['close'].iloc[-1]:.2f}
    Latest RSI: {stock_df['rsi'].iloc[-1]:.1f}
    
    Recent Performance:
    - 5-day change: {stock_df['price_change_5d'].iloc[-1]:.2%}
    - Volume ratio: {stock_df['volume_ratio'].iloc[-1]:.2f}
    """
    
    ax9.text(0.1, 0.9, summary_text, transform=ax9.transAxes, fontsize=11, 
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7))
    ax9.set_xlim(0, 1)
    ax9.set_ylim(0, 1)
    ax9.axis('off')
    
    plt.tight_layout()
    plt.show()

def create_live_chart(stock_df, ticker, market_type):
    """Create a live-updating chart simulation"""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
    
    # Determine currency symbol
    currency = "₹" if market_type == "Indian" else "$"
    
    def animate(frame):
        # Clear previous plots
        ax1.clear()
        ax2.clear()
        
        # Get data up to current frame
        current_data = stock_df.iloc[:frame+1]
        
        if len(current_data) > 0:
            # Plot price data
            ax1.plot(current_data.index, current_data['close'], 'b-', linewidth=2, label='Close Price')
            if len(current_data) >= 5:
                ax1.plot(current_data.index, current_data['ma_5'], 'r--', alpha=0.7, label='MA 5')
            if len(current_data) >= 10:
                ax1.plot(current_data.index, current_data['ma_10'], 'g--', alpha=0.7, label='MA 10')
            
            ax1.set_title(f'{ticker} - Live Price Chart', fontsize=14, fontweight='bold')
            ax1.set_ylabel(f'Price ({currency})')
            ax1.legend()
            ax1.grid(True, alpha=0.3)
            
            # Plot volume data
            ax2.bar(current_data.index, current_data['volume'], alpha=0.7, color='purple')
            ax2.set_title(f'{ticker} - Volume', fontsize=14, fontweight='bold')
            ax2.set_ylabel('Volume')
            ax2.grid(True, alpha=0.3)
            
            # Rotate x-axis labels for better readability
            ax1.tick_params(axis='x', rotation=45)
            ax2.tick_params(axis='x', rotation=45)
    
    # Create animation
    anim = FuncAnimation(fig, animate, frames=len(stock_df), interval=500, repeat=False)
    
    plt.tight_layout()
    plt.show()
    
    return anim

def get_prediction_explanation(prediction, probability):
    """Explain the prediction in human terms"""
    confidence = max(probability) * 100
    
    if prediction == 1:
        direction = "UP ⬆️"
        explanation = "The model predicts the stock will close HIGHER tomorrow"
    else:
        direction = "DOWN ⬇️"
        explanation = "The model predicts the stock will close LOWER tomorrow"
    
    return direction, explanation, confidence

def run_prediction():
    """Main function to run the stock prediction with plotting"""
    print("=== Next-Day Stock Direction Predictor with Plotting ===")
    print("This will predict if the stock will go UP or DOWN tomorrow and create visualizations\n")
    
    # Get company name and ticker
    company_name = input("Enter the company name: ")
    ticker, market_type = get_ticker_by_name(company_name)
    
    if not ticker:
        print("❌ Could not find ticker for this company name.")
        print("\nAvailable Indian stocks:", ', '.join(list(INDIAN_STOCKS.keys())[:10]), "...")
        print("Available US stocks:", ', '.join(list(US_STOCKS.keys())[:10]), "...")
        ticker = input("\nPlease enter the ticker symbol manually (e.g., AAPL or RELIANCE.BSE): ").upper()
        
        # Determine market type from ticker
        if '.BSE' in ticker or '.NSE' in ticker:
            market_type = 'Indian'
        else:
            market_type = 'US'
    else:
        print(f"✅ Found ticker: {ticker} ({market_type} market)")
    
    # Validate ticker
    print("🔍 Validating ticker symbol...")
    if not validate_ticker_symbol(ticker):
        print("❌ Invalid ticker symbol. Please check and try again.")
        return
    
    try:
        # Get stock data (90 days)
        print("\n📊 Fetching stock data...")
        stock_df = get_alpha_vantage_data(ticker, days=90)
        
        # Determine currency for display
        currency = "₹" if market_type == "Indian" else "$"
        
        print(f"✅ Retrieved {len(stock_df)} days of stock data")
        print(f"📈 Price range: {currency}{stock_df['close'].min():.2f} - {currency}{stock_df['close'].max():.2f}")
        
        # Get sentiment data
        print("\n📰 Fetching news sentiment...")
        news_data = get_marketaux_sentiment(ticker, days=7)
        sentiment_score = analyze_sentiment(news_data)
        
        sentiment_emoji = "😊" if sentiment_score > 0.1 else "😐" if sentiment_score > -0.1 else "😟"
        print(f"💭 Sentiment score: {sentiment_score:.3f} {sentiment_emoji} (-1=negative, 0=neutral, 1=positive)")
        
        # Calculate technical indicators
        print("\n🔧 Calculating technical indicators...")
        stock_df = calculate_technical_indicators(stock_df)
        
        # Prepare features
        features_df = prepare_features(stock_df, sentiment_score)
        
        print(f"\n✅ Prepared {len(features_df)} rows of feature data")
        print(f"\n📊 Latest stock data ({currency}):")
        latest_data = stock_df[['open', 'high', 'low', 'close', 'volume']].tail(3)
        print(latest_data.to_string())
        
        # Train model and make prediction
        print("\n🤖 Training model...")
        model, prediction, probability, results_df = train_and_predict_with_history(features_df)
       
           

        
        if model is not None:
            joblib.dump(model, 'stock_model.pkl')
            print("✅ Model saved as stock_model.pkl")
        
            direction, explanation, confidence = get_prediction_explanation(prediction, probability)
            
            print(f"\n🎯 === PREDICTION RESULTS ===")
            print(f"Direction: {direction}")
            print(f"Explanation: {explanation}")
            print(f"Confidence: {confidence:.1f}%")
            print(f"Probability Distribution: Down={probability[0]:.3f}, Up={probability[1]:.3f}")
            
            # Ask user what type of visualization they want
            print(f"\n📊 === VISUALIZATION OPTIONS ===")
            print("1. Comprehensive Analysis Dashboard")
            print("2. Live Chart Animation")
            print("3. Both")
            print("4. Skip plotting")
            
            choice = input("\nEnter your choice (1-4): ")
            
            if choice == "1" or choice == "3":
                print("\n🎨 Generating comprehensive analysis dashboard...")
                plot_comprehensive_analysis(stock_df, results_df, ticker, sentiment_score, market_type)
            
            if choice == "2" or choice == "3":
                print("\n🎬 Generating live chart animation...")
                anim = create_live_chart(stock_df, ticker, market_type)
            
            print(f"\n⚠️  DISCLAIMER: This is for educational purposes only.")
            print(f"   Do not use this for actual trading decisions!")
            print(f"   {market_type} market data is subject to exchange regulations.")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("💡 Suggestions:")
        print("- Check your internet connection")
        print("- Verify API keys are valid")
        print("- Try a different stock symbol")
        print("- Wait a moment if you hit API rate limits")

def interactive_stock_search():
    """Interactive function to help users find stock tickers"""
    print("🔍 === Stock Ticker Search Helper ===")
    print("This will help you find the correct ticker symbol for your stock.\n")
    
    search_term = input("Enter a company name or partial name: ").lower()
    
    found_indian = []
    found_us = []
    
    # Search in Indian stocks
    for company, ticker in INDIAN_STOCKS.items():
        if search_term in company:
            found_indian.append((company, ticker))
    
    # Search in US stocks
    for company, ticker in US_STOCKS.items():
        if search_term in company:
            found_us.append((company, ticker))
    
    if found_indian:
        print("\n🇮🇳 Found Indian stocks:")
        for i, (company, ticker) in enumerate(found_indian, 1):
            print(f"{i}. {company.title()} -> {ticker}")
    
    if found_us:
        print("\n🇺🇸 Found US stocks:")
        for i, (company, ticker) in enumerate(found_us, 1):
            print(f"{i}. {company.title()} -> {ticker}")
    
    if not found_indian and not found_us:
        print(f"❌ No matches found for '{search_term}'")
        print("\nTry searching with different keywords or check our available stocks:")
        print("Indian stocks:", ', '.join(list(INDIAN_STOCKS.keys())[:10]), "...")
        print("US stocks:", ', '.join(list(US_STOCKS.keys())[:10]), "...")

def show_available_stocks():
    """Show all available stocks in the system"""
    print("📋 === Available Stocks ===\n")
    
    print("🇮🇳 Indian Stocks (BSE):")
    print("-" * 50)
    for i, (company, ticker) in enumerate(INDIAN_STOCKS.items(), 1):
        print(f"{i:2d}. {company.title():<25} -> {ticker}")
    
    print(f"\n🇺🇸 US Stocks (NASDAQ/NYSE):")
    print("-" * 50)
    for i, (company, ticker) in enumerate(US_STOCKS.items(), 1):
        print(f"{i:2d}. {company.title():<25} -> {ticker}")
    
    print(f"\nTotal: {len(INDIAN_STOCKS)} Indian stocks, {len(US_STOCKS)} US stocks")

def main_menu():
    """Main menu for the stock prediction system"""
    print("📈 === Stock Prediction System ===")
    print("1. Run Stock Prediction")
    print("2. Search for Stock Ticker")
    print("3. Show Available Stocks")
    print("4. Exit")
    
    choice = input("\nEnter your choice (1-4): ")
    
    if choice == "1":
        run_prediction()
    elif choice == "2":
        interactive_stock_search()
    elif choice == "3":
        show_available_stocks()
    elif choice == "4":
        print("👋 Goodbye!")
        return False
    else:
        print("❌ Invalid choice. Please try again.")
    
    return True

# Run the prediction
if __name__ == "__main__":
    while True:
        try:
            if not main_menu():
                break
            
            print("\n" + "="*60 + "\n")
            
        except KeyboardInterrupt:
            print("\n\n👋 Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ An error occurred: {e}")
            print("Please try again.\n")

📈 === Stock Prediction System ===
1. Run Stock Prediction
2. Search for Stock Ticker
3. Show Available Stocks
4. Exit



Enter your choice (1-4):  1


=== Next-Day Stock Direction Predictor with Plotting ===
This will predict if the stock will go UP or DOWN tomorrow and create visualizations

